In [1]:
import numpy as np
import sys
import ipdb

sys.path.append("../")

sys.path.append("../utils/")
from utils_functions import *

from dd_game import *
from two_stage_player import *

# Configurations

In [2]:
#TODO: There are many configuration parameters here. It is important to understand the effect of all.
d = 2
coeff_var = 1

Sigma_x_p1 = np.eye(d)
Sigma_x_p2 = np.eye(d)

sigma_y_p1 = 0
sigma_y_p2 = 0

beta_p1 = np.random.normal(size = d)
beta_p2 = np.random.normal(size = d)

mu_p1 = np.random.normal(scale = coeff_var, size = d)
mu_p2 = np.random.normal(scale = coeff_var, size = d)
gamma_p1 = np.random.normal(scale = coeff_var, size = d)
gamma_p2 = np.random.normal(scale = coeff_var, size = d)

#Fixing mu and gamma for testing purposes
if False:
    mu_p2 = np.array([2.1, -1.8])
    gamma_p2 = np.array([1.2, 0.9])
    mu_p1 = np.array([-2.8, 2.2])
    gamma_p1 = np.array([0.7, -0.3])

if False:
    mu_p1 = np.array([0, 0])
    gamma_p1 = np.array([1, 1])
    mu_p2 = np.array([0, 0])
    gamma_p2 = np.array([1, 1])

In [3]:
p1_data_params = (Sigma_x_p1, sigma_y_p1, beta_p1, mu_p1, gamma_p1)
p1_data_generating_func = sample_from_location_family

p2_data_params = (Sigma_x_p2, sigma_y_p2, beta_p2, mu_p2, gamma_p2)
p2_data_generating_func = sample_from_location_family

num_rounds_lst = [10, 100, 1000, 10000, 40000]
num_alternate_rounds = 10000
num_test = 10000

# Run Game

In [4]:
#Initialize game with configurations specified above
p1_risks = []
p2_risks = []
theta_final_p1 = []
theta_final_p2 = []


for num_rounds in num_rounds_lst:
    player_two = TwoStagePlayer()
    player_one = TwoStagePlayer()
    game = DecisionDependentGame(player_one, player_two, p1_data_params, p2_data_params, 
                             p1_data_generating_func, p2_data_generating_func, 
                             num_rounds, num_alternate_rounds, num_test)
    
    game.run_train()
    game.run_post_train_alternating()
    
    p1_risks.append(game.evaluate_test_perf_risk_p1())
    p2_risks.append(game.evaluate_test_perf_risk_p2())
    
    theta_final_p1.append(game.theta_p1)
    theta_final_p2.append(game.theta_p2)
    
    #Change this flag if one wants to change print settings
    if True:
        print("---------------------")
        print("Player 1")
        print(mu_p1, gamma_p1)
        print(player_one.mu_hat, player_one.gamma_hat)

        print("Player 2")
        print(mu_p2, gamma_p2)
        print(player_two.mu_hat, player_two.gamma_hat)

Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
---------------------
Player 1
[-1.62006853 -0.64829627] [1.22302896 1.14905918]
[-1.41435178 -0.77266509] [0.94591574 1.23647187]
Player 2
[0.99784833 0.78263443] [-0.27212025  0.31324862]
[0.52644049 0.67019277] [0.64678478 1.05436967]
Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
---------------------
Player 1
[-1.62006853 -0.64829627] [1.22302896 1.14905918]
[-1.67259319 -0.6747885 ] [1.25049589 1.14436032]
Player 2
[0.99784833 0.78263443] [-0.27212025  0.31324862]
[1.11194821 0.45134757] [-0.31247668  0.25667477]
Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
---------------------
Player 1
[-1.62006853 -0.6482962

In [16]:
theta_PO_1, theta_PO_2 = game.solve_nash()
theta_PO_2

array([ 0.50813744, -1.02238872])

In [17]:
theta_final_p2

[array([ 0.04343122, -0.3950157 ]),
 array([ 0.44214491, -1.14367345]),
 array([ 0.49711898, -0.98457521]),
 array([ 0.51962443, -0.99520161]),
 array([ 0.5258376 , -0.99715729])]

In [7]:
beta_p1

array([-0.20040189, -0.46700161])

In [8]:
p1_risks

[0.05208038198988852,
 0.007638845321388945,
 0.004531289965334142,
 0.0035929662129384234,
 0.0038535470013841247]

In [9]:
p2_risks

[0.956080336857491,
 0.3819308789291651,
 0.33778924409747096,
 0.3322599637249413,
 0.34222110487418256]

In [10]:
if False:
    print(mu_p1, gamma_p1)
    print(player_one.mu_hat.round(2), player_one.gamma_hat.round(2))

    print(mu_p2, gamma_p2)
    print(player_two.mu_hat.round(2), player_two.gamma_hat.round(2))

    print(game.solve_nash())
    print(game.solve_social_opt())
    
    print(game.evaluate_test_perf_risk_p1().round(3))
    print(game.evaluate_test_perf_risk_p2().round(3))

## Checking stage 1

In [11]:
print(player_one.mu_hat, player_one.gamma_hat)
print(player_two.mu_hat, player_two.gamma_hat)

[-1.61965815 -0.64970048] [1.2229349  1.14991866]
[0.99892413 0.78700993] [-0.25703513  0.31454662]


In [12]:
print(mu_p1, gamma_p1)
print(mu_p2, gamma_p2)

[-1.62006853 -0.64829627] [1.22302896 1.14905918]
[0.99784833 0.78263443] [-0.27212025  0.31324862]


## Evaluate the performative risks before and after Stage 3

In [13]:
a

NameError: name 'a' is not defined

In [ ]:
PR_1 = game.evaluate_test_perf_risk_p1()
PR_2 = game.evaluate_test_perf_risk_p2()
print(f'Player 1 PR = {PR_1.round(3)}\tPlayer 2 PR = {PR_2.round(3)}')

In [ ]:
print(player_one.theta_history[-1])
print(player_two.theta_history[-1])

In [ ]:
game.theta_p1, game.theta_p2 = game.solve_nash()
PO_1 = game.evaluate_test_perf_risk_p1()
PO_2 = game.evaluate_test_perf_risk_p2()
print(f'Player 1 PO = {PO_1.round(3)}\tPlayer 2 PO = {PO_2.round(3)}')

In [ ]:
game.theta_p1, game.theta_p2 = game.solve_social_opt()
SO_1 = game.evaluate_test_perf_risk_p1()
SO_2 = game.evaluate_test_perf_risk_p2()
print(f'Player 1 SO = {SO_1.round(3)}\tPlayer 2 SO = {SO_2.round(3)}')